In [15]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["OPENAI_API_KEY"] = "..."

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [3]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load_and_split()

In [4]:
def create_vector_store(_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(_docs)
    persist_directory = "./chroma_db"
    vectorstore = Chroma.from_documents(
        split_docs, 
        OpenAIEmbeddings(model='text-embedding-3-small'),
        persist_directory=persist_directory
    )
    return vectorstore

In [5]:
def add_to_vector_store(new_docs, persist_directory="./chroma_db"):
    # 1. 기존 벡터 저장소 불러오기
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(new_docs)

    # 3. 문서 추가
    vectorstore.add_documents(split_docs)

    return vectorstore

In [8]:
path='C:/python workspace/3-3/챗봇/논문/지준화_학위.pdf'

In [9]:
sp=load_and_split_pdf(path)

In [12]:
add_to_vector_store(sp)

In [13]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "문형빈 교수님의 석사 주제는?"
result = db3.similarity_search(query)
print(result[0].page_content)

공학 석사학위 논문 
 
 
제품유형별 소비자 정보탐색 행동 분석 
- 탐색재와 경험재를 중심으로 - 
 
Analysis on Consumer Information Search Behavior  
for Search and Experience Goods 
 
 
 
 
2014 년 2 월 
 
 
 
서울대학교 대학원  
협동과정 기술경영경제정책전공  
문   형   빈


In [14]:
query = "지준화 교수님의 석사 주제는?"
result = db3.similarity_search(query)
print(result[0].page_content)

工學碩士學位論文
KOMPSAT-2 고해상도 위성영상의
절대복사보정과 검증
Absolute radiometric calibration and validation
of KOMPSAT-2 high resolution images
2008年 8月
指導敎授 李 奎 成
이 論文을 工學碩士學位 論文으로 提出함
仁荷大學校 大學院
地理情報工學科
智 俊 華
